In [1]:
!git clone https://github.com/sk27110/ImageCaption.git

Cloning into 'ImageCaption'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 23 (delta 8), reused 23 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 115.03 KiB | 1.01 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [6]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.9 MB/s eta 0:00:00


In [7]:
!pip install comet_ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.8 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.3.2
    Uninstalling python-box-7.3.2:
      Successfully uninstalled python-box-7.3.2


In [8]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 138.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
from comet_ml import API
from pathlib import Path

api = API(api_key="MbL2psOHT82Uc7ML5Cd7TSvmR")

workspace = "-3767"
project_name = "image_captioning"
experiment_key = "392a142eb5be49989142077df24d091d"

exp = api.get_experiment(workspace, project_name, experiment_key)

output_dir = Path("./from_experiment_assets")
output_dir.mkdir(exist_ok=True)

for asset in exp.get_asset_list():
    fname = asset["fileName"]
    if "best_model_latest" in fname:
        binary = exp.get_asset(asset["assetId"], return_type="binary")

        with open(output_dir / fname, "wb") as f:
            f.write(binary)

        print("✅ Скачан:", fname)


✅ Скачан: best_model_latest.pth


In [10]:
!ls

from_experiment_assets	logs.log  README.md	    src
HW5.pdf			main.py   requirements.txt  test.ipynb


In [11]:
from src.dataset import get_datasets

In [12]:
train , _, test = get_datasets()

Using Colab cache for faster access to the 'flickr8k' dataset.
Path to dataset files: /kaggle/input/flickr8k


In [13]:
for asset in exp.get_asset_list():
    fname = asset["fileName"]
    # print(fname)
    if "tokenizer" in fname:
        binary = exp.get_asset(asset["assetId"], return_type="binary")

        with open(output_dir / fname, "wb") as f:
            f.write(binary)

        print("✅ Скачан:", fname)

✅ Скачан: tokenizer.pth


In [14]:
from src.clip_transformer_model import CLIPTransformerEncoderDecoder
import torch

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load("/content/ImageCaption/from_experiment_assets/best_model_latest.pth", map_location=device)
model = CLIPTransformerEncoderDecoder(**checkpoint["model_args"]).to(device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

CLIPTransformerEncoderDecoder(
  (encoder): Encoder(
    (vision_model): CLIPVisionModel(
      (vision_model): CLIPVisionTransformer(
        (embeddings): CLIPVisionEmbeddings(
          (patch_embedding): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
          (position_embedding): Embedding(50, 768)
        )
        (pre_layrnorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (encoder): CLIPEncoder(
          (layers): ModuleList(
            (0-11): 12 x CLIPEncoderLayer(
              (self_attn): CLIPAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
                (v_proj): Linear(in_features=768, out_features=768, bias=True)
                (q_proj): Linear(in_features=768, out_features=768, bias=True)
                (out_proj): Linear(in_features=768, out_features=768, bias=True)
              )
              (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              (mlp): 

In [17]:
from src.dataset import Vocabulary

In [18]:
checkpoint = torch.load("/content/ImageCaption/from_experiment_assets/tokenizer.pth", map_location=device)

In [19]:
itos = checkpoint['itos']
stoi = checkpoint['stoi']

In [20]:
input = test[0][0].unsqueeze(0).to(device)

In [21]:
gen_ids=model.generate(input)[0]

In [22]:
gen_tokens = [itos[idx] for idx in gen_ids if idx != stoi["<PAD>"]]

In [23]:
gen_tokens

['<START>', 'two', 'people', 'sitting', 'on', 'a', 'bench', '.', '<END>']

In [24]:
!pip install pycocoevalcap
!pip install pycocotools

In [25]:
from tqdm import tqdm
from pycocoevalcap.cider.cider import Cider

In [38]:
pred_caption = dict()
true_captions = dict()

for img, captions, img_name in tqdm(test):
    input = img.unsqueeze(0).to(device)
    gen_ids = model.generate_beam(
                                input,
                                max_len=25,
                                beam_width=3,
                                start_token=stoi["<START>"],
                                end_token=stoi["<END>"]
                            )[0]
    gen_tokens = [itos[idx] for idx in gen_ids if idx != stoi["<PAD>"]]
    clear_tokens = []
    for token in gen_tokens:
        if token == '<START>' or token=='<END>':
          continue
        clear_tokens.append(token)

    pred_caption[img_name] = [" ".join(clear_tokens)]
    true_captions[img_name] = captions


100%|██████████| 810/810 [01:24<00:00,  9.54it/s]


In [39]:
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.spice.spice import Spice
from pycocoevalcap.cider.cider import Cider

scorers = [
    (Bleu(4), ["BLEU-1", "BLEU-2", "BLEU-3", "BLEU-4"]),
    (Meteor(), "METEOR"),
    (Rouge(), "ROUGE-L"),
    (Cider(), "CIDEr"),
]

for scorer, name in scorers:
    score, scores = scorer.compute_score(true_captions, pred_caption)

    if isinstance(name, list):
        print(f"{name[3]}: {score[3]:.3f}")
    else:
        print(f"{name}: {score:.3f}")

{'testlen': 7995, 'reflen': 8057, 'guess': [7995, 7185, 6375, 5565], 'correct': [5495, 2310, 934, 326]}
ratio: 0.9923048280996658
BLEU-4: 0.207
METEOR: 0.228
ROUGE-L: 0.480
CIDEr: 0.542
